In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


In [ ]:
# Mount Google Drive if not done already
from google.colab import drive
drive.mount('/content/drive')

# Load your cleaned data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleaned_data.csv')

# Drop unnecessary columns (check your file for exact names)
cols_to_drop = ['Unnamed: 0', 'attacking_work_rate_encoded', 'defensive_work_rate_encoded']
data = data.drop(columns=[col for col in cols_to_drop if col in data.columns])

# Features and target
y = data['player_role']
X = data.drop(columns=['player_role'])


ValueError: mount failed

In [ ]:
# Mount Google Drive if not done already
from google.colab import drive
drive.mount('/content/drive')

# Load your cleaned data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleaned_data.csv')

# Drop unnecessary columns (check your file for exact names)
cols_to_drop = ['Unnamed: 0', 'attacking_work_rate_encoded', 'defensive_work_rate_encoded']
data = data.drop(columns=[col for col in cols_to_drop if col in data.columns])

# Features and target
y = data['player_role']
X = data.drop(columns=['player_role'])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
dt = DecisionTreeClassifier(random_state=42, max_depth=8)  # You can tune max_depth

# Train
dt.fit(X_train_scaled, y_train)

# Predict
y_pred_dt = dt.predict(X_test_scaled)
print()

# Evaluate
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred_dt)*100:.4f}\n")
print("Decision Tree Classification Report:\n" + classification_report(y_test, y_pred_dt) + "\n")
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(pd.DataFrame(cm_dt, index=dt.classes_, columns=dt.classes_), annot=True, fmt='d', cmap='crest')
plt.title('Decision Tree Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
print()

# [Optional] Visualize the tree structure
plt.figure(figsize=(18,8))
plot_tree(dt, feature_names=X.columns, class_names=[str(c) for c in dt.classes_], filled=True, max_depth=2)
plt.title('Top Levels of Decision Tree')
plt.show()
print()

# Save the model
joblib.dump(dt, 'decision_tree_model.pkl')
pd.Series(y_pred_dt).to_csv('decision_tree_y_pred.csv', index=False)

In [ ]:
y_pred_1 = dt.predict(X_test_scaled[1].reshape(1, -2))
print(f"Predicted player role for X_test_scaled[1]: {y_pred_1[0]}")


In [ ]:
y_pred1=dt.predict(X_test_scaled[0].reshape(1, -2))
print(y_pred1)

TEST

In [ ]:
new_player_dict = {
    'crossing': [75],
    'finishing': [82],
    'heading_accuracy': [78],
    'short_passing': [85],
    'volleys': [71],
    'dribbling': [88],
    'curve': [74],
    'free_kick_accuracy': [65],
    'long_passing': [77],
    'ball_control': [80],
    'acceleration': [86],
    'sprint_speed': [89],
    'agility': [85],
    'reactions': [78],
    'balance': [79],
    'shot_power': [80],
    'jumping': [70],
    'stamina': [83],
    'strength': [72],
    'long_shots': [73],
    'positioning': [81],
    'vision': [80],
    'penalties': [68],
    'aggression': [74],
    'interceptions': [77],
    'marking': [72],
    'standing_tackle': [69],
    'sliding_tackle': [68],
    'gk_diving': [10],
    'gk_handling': [12],
    'gk_kicking': [11],
    'gk_positioning': [9],
    'gk_reflexes': [13],
    'preferred_foot_encoded': [1]  # 0 for left, 1 for right
}

new_player_df = pd.DataFrame(new_player_dict)


In [ ]:
import joblib
scaler = joblib.load('scaler.pkl')  # Load the actual scaler object
new_player_scaled = scaler.transform(new_player_df)

In [ ]:
dt = joblib.load('random_forest_model.pkl')  # Use your actual RF model file
pred_encoded = dt.predict(new_player_scaled)[0]
print('Predicted role code:', pred_encoded)


In [ ]:
y_pred1=dt.predict(X_test_scaled[1].reshape(1, -1))
print(y_pred1)

In [ ]:
role_mapping = {
    0: 'GK', 1: 'CB', 2: 'FB', 3: 'DM', 4: 'CM', 5: 'AM', 6: 'W', 7: 'ST'
}
role_label = role_mapping[pred_encoded]
print('Predicted role label:', role_label)


**RandomForest**

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)  # Tune these!

# Train
rf.fit(X_train_scaled, y_train)

# Predict
y_pred_rf = rf.predict(X_test_scaled)

# Evaluate
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)*100:.4f}\n")
print("Random Forest Classification Report:\n" + classification_report(y_test, y_pred_rf) + "\n")
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(pd.DataFrame(cm_rf, index=rf.classes_, columns=rf.classes_), annot=True, fmt='d', cmap='crest')
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plot Feature Importances
importances = pd.Series(rf.feature_importances_, index=X.columns)
plt.figure(figsize=(12,6))
importances.sort_values(ascending=False).plot.bar()
plt.title('Random Forest Feature Importances')
print()
plt.tight_layout()
print()
plt.show()

# Save the model
joblib.dump(rf, 'random_forest_model.pkl')
pd.Series(y_pred_rf).to_csv('random_forest_y_pred.csv', index=False)

In [ ]:
from sklearn.preprocessing import label_binarize
y_test_bin = label_binarize(y_test, classes=rf.classes_)
probs_rf = rf.predict_proba(X_test_scaled)
n_classes = len(rf.classes_)

plt.figure(figsize=(10,8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], probs_rf[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {rf.classes_[i]} (AUC = {roc_auc:.2f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve: Random Forest')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
# Predict role for the first sample in X_test (scaled)
y_pred_rf_0 = rf.predict(X_test_scaled[0].reshape(1, -1))
print(f"Predicted player role for X_test_scaled[0]: {y_pred_rf_0[0]}")

# Predict role for the second sample
y_pred_rf_1 = rf.predict(X_test_scaled[1].reshape(1, -1))
print(f"Predicted player role for X_test_scaled[1]: {y_pred_rf_1[0]}")


In [ ]:
# Compare to true role (encoded value) for first and second samples
print(f"Actual player role for X_test[0]: {y_test.iloc[0]}")
print(f"Actual player role for X_test[1]: {y_test.iloc[1]}")


In [ ]:
role_mapping = {0: 'GK', 1: 'CB', 2: 'FB', 3: 'DM', 4: 'CM', 5: 'AM', 6: 'W', 7: 'ST'}

print(f"Predicted label for X_test[0]: {role_mapping[y_pred_rf_0[0]]}")
print(f"Actual label for X_test[0]: {role_mapping[y_test.iloc[1]]}")
